# AI Judge pre-processing - skillborders

In [1]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import cv2 # pip install opencv-python

# constanten

In [3]:
video_border_labels_path = 'data/df_video_border_labels.pkl'

train_videos = [
    'videos/20240201_atelier_001.mp4',
    'videos/20240201_atelier_002.mp4',
    'videos/20240201_atelier_003.mp4',
    'videos/20240201_atelier_004.mp4',
    'videos/20240201_atelier_005.mp4',
    'videos/20240209_atelier_006.mp4',
    'videos/20240209_atelier_007.mp4',
    'videos/20240209_atelier_008.mp4',
]

# functies

In [4]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [5]:
def get_random_frame(videos, framelabels):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
        [ 0 : ground
          1 : heels of ground
          2 : air ]
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # print(len(framelabels[framelabels['path'] == path]) / video_length) 
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [6]:
def show_frame(frame, scale=0.4):
    """
    Displays a single frame using OpenCV, waits for a key press, 
    and then closes the frame window.

    Args:
        frame (numpy.ndarray): The image frame to display.

    Returns:
        int: The ASCII value of the key pressed.
    """
    if scale != 1.0:
        frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # Display the frame using OpenCV
    cv2.imshow('Frame', frame)
    
    # Wait for a key press
    key = cv2.waitKey(0)
    
    return key

In [7]:
def match_label(key):
    match key:
        case 'n': 
            return 9
        case 'f':
            return 5
        case 'a' | '2':
            return 2
        case 'h' | 's' | 'm' | '1':
            return 1
        case 'g' | '0':
            return 0

def is_already_labeled(path, frame_nr, df_labels):
    return len(df_labels[(df_labels['path'] == path) & (df_labels['frame'] == frame_nr)]) > 0

def label_frames(df_labels):
    """
    Q : Quit (

    0, g = ground (48)
    1, s, m, h = start of multiple, heels (49)
    2, a = air (50)
    n = 9 = no skipper or not skipping
    f = fout
    """
    allowed_keys = ['0', '1', '2', 'n', 'a', 'm', 's', 'h', 'g', 'f']
    quit_key = 'q'
    key_pressed = 'none'
    path, frame_nr, frame = get_random_frame(train_videos, df_video_border_labels)
    while key_pressed != quit_key:
        while is_already_labeled(path, frame_nr, df_labels):
            path, frame_nr, frame = get_random_frame(train_videos, df_video_border_labels)
        
        key_pressed = chr(show_frame(frame, 0.5))
        if key_pressed in allowed_keys:
            label = match_label(key_pressed)
            df_labels.loc[len(df_labels)] = [path, frame_nr, label]

    # Close the frame window
    cv2.destroyAllWindows()

In [14]:
df_video_border_labels = pickle_load_or_create(video_border_labels_path, ['path', 'frame', 'border'])
df_video_border_labels.sort_values(['path', 'frame'])

,path,frame,border
370,videos/20240201_atelier_001.mp4,3,9
96,videos/20240201_atelier_001.mp4,14,9
298,videos/20240201_atelier_001.mp4,18,9
110,videos/20240201_atelier_001.mp4,28,9
616,videos/20240201_atelier_001.mp4,30,9
...,...,...,...
527,videos/20240209_atelier_007.mp4,1252,9
563,videos/20240209_atelier_007.mp4,1270,9
174,videos/20240209_atelier_007.mp4,1281,9
609,videos/20240209_atelier_007.mp4,1333,9


In [9]:
label_frames(df_video_border_labels)

In [10]:
df_video_border_labels.sort_values(['path', 'frame'])

,path,frame,border
370,videos/20240201_atelier_001.mp4,3,9
96,videos/20240201_atelier_001.mp4,14,9
298,videos/20240201_atelier_001.mp4,18,9
110,videos/20240201_atelier_001.mp4,28,9
616,videos/20240201_atelier_001.mp4,30,9
...,...,...,...
527,videos/20240209_atelier_007.mp4,1252,9
563,videos/20240209_atelier_007.mp4,1270,9
174,videos/20240209_atelier_007.mp4,1281,9
609,videos/20240209_atelier_007.mp4,1333,9


In [18]:
with open(video_border_labels_path, 'wb') as handle:
    pickle.dump(df_video_border_labels, handle)

In [19]:
pd.set_option('display.max_rows', None)
df_video_border_labels.sort_values(['path', 'frame'])

,path,frame,border
370,videos/20240201_atelier_001.mp4,3,9
96,videos/20240201_atelier_001.mp4,14,9
298,videos/20240201_atelier_001.mp4,18,9
110,videos/20240201_atelier_001.mp4,28,9
616,videos/20240201_atelier_001.mp4,30,9
356,videos/20240201_atelier_001.mp4,35,9
555,videos/20240201_atelier_001.mp4,40,9
602,videos/20240201_atelier_001.mp4,50,9
395,videos/20240201_atelier_001.mp4,59,9
557,videos/20240201_atelier_001.mp4,64,0


In [20]:
pd.set_option('display.max_rows', 30)

In [22]:
df_video_border_labels.border.value_counts()

border
0    322
2    162
9    116
1     52
5      6
Name: count, dtype: int64